In [1]:
import talib as taLib
import pymongo
import pandas as pd
import numpy as np
import nsepy
import math
from datetime import datetime
from datetime import timedelta
from datetime import date
from pymongo import MongoClient
from nsepy import get_history
from TAcharts.indicators.ichimoku import Ichimoku
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.shape[0]

8425

In [2]:
print(equityListPd.columns)
equityListPd.head(5)

Index(['Face Value', 'Group', 'ISIN No', 'Industry', 'Issuer Name',
       'Security Code', 'Security Id', 'Security Name', 'State', 'Status',
       '_id', 'optionsTrade'],
      dtype='object')


,Face Value,Group,ISIN No,Industry,Issuer Name,Security Code,Security Id,Security Name,State,Status,_id,optionsTrade
0,INE117A01022,2,Heavy Electrical Equipment,Equity,ABB,500002,ABB India Limited,Active,A,B,5e6dffa36cef0bbb3f7833ea,NaN
1,INE424A01014,10,,Equity,TPAEC,500004,TORRENT POWER AEC LTD.,Delisted,NaN,B,5e6dffa36cef0bbb3f7833eb,NaN
2,INE885A01032,1,Auto Parts & Equipment,Equity,AMARAJABAT,500008,AMARA RAJA BATTERIES LTD.,Active,A,A,5e6dffa36cef0bbb3f7833ec,A
3,INE984C01013,10,Iron & Steel Products,Equity,AKARLAMIN,500005,AKAR LAMINATORS LTD.,Delisted,NaN,XD,5e6dffa36cef0bbb3f7833ed,NaN
4,INE001A01036,2,Housing Finance,Equity,HDFC,500010,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Active,A,A,5e6dffa36cef0bbb3f7833ee,A


In [3]:
def dataCollector():
    equityListPd = pd.DataFrame(equityList.find({'Security Name':'Active','State':"A"}))
    collectionTime=2000
    basicSample=30
    cnt=0
    my_time = datetime.now()

    for rows in equityListPd['Issuer Name'].iteritems():
        print(cnt,rows[1],end="-")
        cnt= cnt + 1
        if stockWarehouse.find({'Symbol':rows[1]}).count() > 1 :
            for j in stockWarehouse.find({'Symbol':rows[1]},{'_id':0,'Date':1}).sort('Date',-1).limit(1) :
                collectionTime=((my_time - j['Date']).days - 1)
                fromDate=j['Date']
        iters=int(collectionTime/basicSample)
        if (collectionTime != 0):
            if collectionTime < 2000:
                basicSample=collectionTime
                iters=1
            dt=datetime.now()
            print("Iterations:",iters,end="|")
            for i in range(0,iters):

                startDate=dt - timedelta(days=basicSample)
                if( startDate <= fromDate):
                    startDate = (fromDate + timedelta(days=1))
                endDate=dt 
                print(i,"Collecting into between",startDate.date(),"-", endDate.date(),"@",datetime.now(),end="|")
                df=pd.DataFrame(get_history(symbol=rows[1] , start= startDate,end=endDate))
                df.reset_index(inplace=True)
                df['Date']=pd.to_datetime(df['Date'])
                dfLength=(len(df.index))
                if  dfLength > 0 :
                    data_dict=df.to_dict("records")
                    stockWarehouse.insert_many(data_dict)
                    tmpdf=pd.DataFrame(stockWarehouse.find({"Symbol":rows[i],"Date":{"$lte":endDate,"$gte":startDate}}))
                    print(df.index.size,tmpdf.index.size,end="|")

                else :
                    print("Marking as Inactive")
                    equityList.update_many({'Issuer Name':rows[1]},{'$set': {'Security Name':'Inactive'}})
                    break;
                dt=(startDate - timedelta(days=1))


In [4]:
def checkType(x):
    if x['Close']>x['Open']:
        return 1
    else :
        return -1
def pullbackTrendIdentify(df):
    df['type']=df.apply(lambda x: checkType(x),axis=1)
    length=df.shape[0] -1
    uptrenddaycnt=0
    bearcnt=0
    slope=0
    stoploss=0
    target=0
    signal=0
                
    if df['type'].iloc[length] > 0:
        for i in range(length-1,1,-1):
            if df['type'].iloc[i] < 0 :
                bearcnt=bearcnt+1
            else:
                break;
        if bearcnt > 2:
            (macd,macdsignal,macdhistory)=(taLib.MACD(datadf['Close'], fastperiod=26, slowperiod=9, signalperiod=9))
            df['macd']=macd
            for i in range((length - 1) - bearcnt,1,-1):
                if df['macd'].iloc[i] >0:
                    uptrenddaycnt=uptrenddaycnt+1
                else:
                    break;
            if uptrenddaycnt > 1:
                high=df['High'].iloc[(length-uptrenddaycnt):length].max()
                low=df['Low'].iloc[(length-uptrenddaycnt):length].min()
                slope=math.degrees(math.atan((high-low)/uptrenddaycnt))
                signal=1
            if signal==1:
                stoploss=df['Low'].iloc[(length -1) - bearcnt - uptrenddaycnt: length - 1].min()
                target=df['Close'].iloc[length] *1.05
    return (df,slope,bearcnt,uptrenddaycnt,stoploss,target,signal,df['Close'].iloc[length-1])
    



In [5]:
equityListdf=pd.DataFrame(equityList.find({'State':"A"},{'Issuer Name':1,'_id':0},no_cursor_timeout=True))
equityListdf['trend']=0
equityListdf['slope']=0
equityListdf['bearcnt']=0
equityListdf['uptrenddaycnt']=0
equityListdf['stoploss']=0
equityListdf['target']=0
equityListdf['signal']=0
equityListdf['Close']=0
equityListdf['sharesbought']=0
equityListdf['moneyinvested']=0    
equityListdf['profit']=0
equityListdf['trades']=0
capital_needed=0
portfolio=np.array([['Symbol','price','number of shares']])
equityListdf.columns

Index(['Issuer Name', 'trend', 'slope', 'bearcnt', 'uptrenddaycnt', 'stoploss',
       'target', 'signal', 'Close', 'sharesbought', 'moneyinvested', 'profit',
       'trades'],
      dtype='object')

In [ ]:
capital_needed=0
equityListdf['trend']=0
equityListdf['slope']=0
equityListdf['bearcnt']=0
equityListdf['uptrenddaycnt']=0
equityListdf['stoploss']=0
equityListdf['target']=0
equityListdf['signal']=0
equityListdf['Close']=0
equityListdf['sharesbought']=0
equityListdf['moneyinvested']=0    
equityListdf['profit']=0
equityListdf['trades']=0
capital_needed=0
portfolio=np.array([['Symbol','price','number of shares']])

for dt in stockWarehouse.find({'Symbol':'SBIN','Date':{'$gte':datetime(2020,6,1)}},{'_id':0,'Date':1},no_cursor_timeout=True).sort('Date'):
    buy=0
    sell=0
    profit=0  
    trades=0
    sell=0
    buy=0
    limit=50000
    for symbol in equityListdf['Issuer Name']:

        datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol,'Date':{'$lte':dt['Date']}},{'_id':0,'%Deliverable':1,'Close':1,'Open':1,'High':1,'Low':1,'Date':1}).sort('Date',-1).limit(90))
        datadf.set_index('Date', inplace=True)
        datadf.sort_index(ascending=True,inplace=True)
        (datadf,slope,bearcnt,uptrenddaycnt,stoploss,target,signal,close)=pullbackTrendIdentify(datadf)
        
        equityListdf.loc[equityListdf['Issuer Name']==symbol,'Close']=close
        
        if signal > 0 and slope > 30 and slope < 50 and capital_needed < 50000:
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'trend']=datadf['macd'].iloc[datadf.shape[0]-1]
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'slope']=slope
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'bearcnt']=bearcnt
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'uptrenddaycnt']=uptrenddaycnt
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'stoploss']=stoploss
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'target']=target
            tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy()
        
            for cls in stockWarehouse.find({'Symbol':symbol,'Date':{'$gt':dt['Date']}},{'_id':0,'Open':1,'Date':1}).sort('Date',1).limit(1):
                buyprice=(cls['Open'] * 1.005)
            tempnp[0,9]=tempnp[0,9] + round(2500/buyprice)            
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'sharesbought']=tempnp[0,9]
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'moneyinvested']=tempnp[0,10] + (tempnp[0,8]*round(2500/buyprice))
            capital_needed=capital_needed+(tempnp[0,8]*round(2500/buyprice))
            equityListdf.loc[equityListdf['Issuer Name']==symbol,'trades']=tempnp[0,12]+1
            buy=buy+1
            portfolio=np.vstack([portfolio,[[symbol,tempnp[0,8],tempnp[0,9]]]])
            print('          BUY-',symbol,equityListdf.loc[equityListdf['Issuer Name']==symbol,'moneyinvested'].to_string())
        else:
            tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy()
        
            if tempnp[0,9] > 0:
                avgprice=tempnp[0,10]/tempnp[0,9]
                if tempnp[0,8] < tempnp[0,5]  or tempnp[0,8] > tempnp[0,6]:
                    tempnp[0,11]=tempnp[0,11] + (tempnp[0,8]*tempnp[0,9] - tempnp[0,10])
                    equityListdf.loc[equityListdf['Issuer Name']==symbol,'profit']=tempnp[0,11]
                    equityListdf.loc[equityListdf['Issuer Name']==symbol,'moneyinvested']=0
                    equityListdf.loc[equityListdf['Issuer Name']==symbol,'trades']=tempnp[0,12]-1
                    equityListdf.loc[equityListdf['Issuer Name']==symbol,'sharesbought']=0
                    capital_needed=capital_needed-tempnp[0,10]
                    sell=sell+1
                    print('          SELL-',symbol,tempnp[0,8],tempnp[0,5],tempnp[0,6])
                
    print(dt['Date'],equityListdf['profit'].sum(),capital_needed,buy,sell)

2020-06-01 00:00:00 0 0 0 0
          BUY- NOCIL 165    2430.4
          BUY- HIKAL 415    2310.0
2020-06-02 00:00:00 0 4740.4 2 0
          BUY- KABRAEXTRU 399    2435.35
          BUY- MCDHOLDING 681    2373.5
          BUY- ARIES 711    2322.25
2020-06-03 00:00:00 0 11871.5 3 0
          BUY- TAJGVK 541    2524.5
          BUY- DLINKINDIA 747    2508.8
2020-06-04 00:00:00 0 16904.8 2 0
          BUY- DEEPAKFERT 155    2354.05
          BUY- WELSPUNIND 304    2428.2
          BUY- DREDGECORP 387    2477.2
          BUY- REDINGTON 659    2378.6
          BUY- AMDIND 670    2385.0
          BUY- EDELWEISS 702    2334.8
          BUY- MOIL 782    2211.7
2020-06-05 00:00:00 0 33474.35 7 0
          SELL- KABRAEXTRU 49.95 44.0 48.5625
          SELL- TAJGVK 165.5 112.0 146.26500000000001
          BUY- AURIONPRO 609    2360.8
          SELL- MCDHOLDING 26.65 17.5 25.8825
          SELL- DLINKINDIA 85.0 70.1 82.425
2020-06-08 00:00:00 1195.8499999999995 25993.0 1 4
          BUY- SMSPHARMA

          BUY- KOTAKNIFTY 834    2381.61
2020-08-04 00:00:00 18343.5 50907.46000000001 1 1
          SELL- NCLIND 83.15 62.25 82.4775
          BUY- SUPERHOUSE 374    2374.4
2020-08-05 00:00:00 18508.300000000003 50785.86000000001 1 1
          SELL- PONNIERODE 160.8 112.1 159.495
          SELL- IMFA 185.8 123.0 182.91
          BUY- OIL 742    2445.3
          BUY- ORIENTREF 812    2402.4
          SELL- WONDERLA 138.3 122.25 138.0225
          BUY- IMPAL 862    2600.5
2020-08-06 00:00:00 19517.500000000004 46267.56000000001 3 3
          BUY- APCOTEXIND 391    2272.0
          BUY- MANAPPURAM 466    2322.0
          SELL- PFC 86.25 80.25 85.3125
2020-08-07 00:00:00 19725.200000000004 48395.51000000001 2 1
          BUY- BEL 15    2281.6
2020-08-10 00:00:00 19725.200000000004 50677.11000000001 1 0
          SELL- MOTHERSUMI 104.9 72.0 104.79
          SELL- IBREALEST 57.3 44.5 55.9125
          SELL- EMMBI 73.8 56.3 73.71000000000001
          BUY- PRICOLLTD 847    2556.4
2020-08-11 

          BUY- JAMNAAUTO 343    2395.0
          SELL- ALLCARGO 126.9 92.15 124.95
2020-11-17 00:00:00 25495.310000000005 47979.99999999999 1 1
          BUY- NAHARSPING 95    2351.9
2020-11-18 00:00:00 25495.310000000005 50331.899999999994 1 0
          SELL- JAMNAAUTO 53.8 43.05 52.1325
          BUY- TFCILTD 431    2443.05
          SELL- WELCORP 109.45 99.0 108.57000000000001
          BUY- INDOCO 586    2539.5
2020-11-19 00:00:00 26003.910000000003 50506.25 2 2
          SELL- INDIACEM 150.45 97.0 134.6625
          SELL- CGCL 277.15 211.2 263.97
          BUY- BROOKS 805    2394.6
2020-11-20 00:00:00 27592.460000000003 43142.25 1 2
          BUY- WALCHANNAG 248    2428.8
          BUY- PIONEEREMB 308    2396.1
          SELL- WINDMACHIN 14.3 13.35 14.07
          BUY- MAGMA 397    2335.5
          BUY- PNBGILTS 535    2288.1
          SELL- VGUARD 176.9 164.05 176.19000000000003
          BUY- FCL 795    2133.6
2020-11-23 00:00:00 28027.510000000002 49946.2 5 2
          BUY- TAT

In [10]:
equityListdf.loc[(equityListdf['profit']>0)]

,Issuer Name,trend,slope,bearcnt,uptrenddaycnt,stoploss,target,signal,Close,sharesbought,moneyinvested,profit,trades


In [10]:
datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol,'Date':{'$lte':dt['Date']}},{'_id':0,'%Deliverable':1,'Close':1,'Open':1,'High':1,'Low':1,'Date':1}).sort('Date',-1).limit(90)

SyntaxError: unexpected EOF while parsing (<ipython-input-10-549ebb6fb49c>, line 1)